In [1]:
#! pip install ipython-sql 
import sqlite3
import pandas as pd
from sqlalchemy import *
from gradedb import Gradedb
from schema import *

In [2]:
%load_ext sql
%sql sqlite:///gradedb.db
engine = create_engine("sqlite:///gradedb.db")

In [4]:
%%sql
SELECT *
  FROM Student

 * sqlite:///gradedb.db
Done.


universityid,name,email
S0416410,Marjorie Steele,s0416410@vuw.leidenuniv.nl
S0225853,James Morse,s0225853@vuw.leidenuniv.nl
S1961057,Dominic Robinson,s1961057@vuw.leidenuniv.nl
S2204697,Diana Brown,s2204697@vuw.leidenuniv.nl
S4587398,Mary Bakley,s4587398@vuw.leidenuniv.nl


In [3]:
%%sql
SELECT *
  FROM Assignment

 * sqlite:///gradedb.db
Done.


assignmentid,universityid,taskid
1,S0416410,1
2,S0225853,2
3,S1961057,3
4,S2204697,4
5,S4587398,5


In [7]:
  
%%sql
SELECT *
  FROM Task

 * sqlite:///gradedb.db
Done.


taskid,title,content
1,Task 688,"This is task 688, there are multiple questions you need answer."
2,Task 405,"This is task 405, there are multiple questions you need answer."
3,Task 871,"This is task 871, there are multiple questions you need answer."
4,Task 980,"This is task 980, there are multiple questions you need answer."
5,Task 913,"This is task 913, there are multiple questions you need answer."


In [8]:
  
%%sql
SELECT *
  FROM Task_question

 * sqlite:///gradedb.db
Done.


id,taskid,questionid
1,1,1
2,1,5
3,1,2
4,2,13
5,2,7
6,2,5
7,3,2
8,3,14
9,3,1
10,4,7


In [5]:
%%sql
SELECT *
  FROM Question

 * sqlite:///gradedb.db
Done.


questionid,title,content
1,Question 1,Over telegrammic unfalteringly gangbangers exudation birken?
2,Question 2,Calculate Hussain brume protochordate None tackie?
3,Question 3,Calculate solariums None inebriation imprescience buddhadharma?
4,Question 4,Over broadish woodlot lomatia wool-gathering None?
5,Question 5,Over photophil None nontheist slugly None?
6,Question 6,How record-holder umbras benk indirubin None?
7,Question 7,Over abode coalition trolling busser bacitracin?
8,Question 8,Calculate celeus blossoming None unsexualized hard-surfaced?
9,Question 9,Calculate ittself LYNDEN None newswire ex-secret?
10,Question 10,How dolomitization carabus channel-surf razzling None?


In [6]:
  
%%sql
SELECT *
  FROM Answers


 * sqlite:///gradedb.db
Done.


answerid,content,questionId,submissionid
1,Given shoobies Seppo grepped constrictive riskful unshatterable Rubbia assmunches.,5,1
2,Given tail-pipe gaminess dabbled whirry Sydneyan None spectacle notating.,5,1
3,Given risoluto factum personate metamorphose henen gin-pole declaiming None.,2,3
4,To calculate MOAS ohelo urd coalfield chapelet characins opinion None.,5,1
5,Because camerata trinucleate arrack lewd gaunt southward concaving None.,1,5
6,Given Galway Bay chakravartin turbocharging heartiest dictated assert anti-imperialism None.,14,3
7,To calculate hell-for-leather cross-mate diuturnal tosylate trick polynucleotide acacia None.,1,1
8,Given misaddressed Begonia Makua loosens BEDMAS Huns tongue-bar None.,7,2
9,Resulting in pincase fluorochemicals Aktyubinsk vitiosities covered jet-black confounder achondritic.,15,5
10,Given conventicler retene assertion copolymers hammams garaging None cobalamin.,13,2


In [9]:
%%sql
SELECT *
  FROM Submission
  

 * sqlite:///gradedb.db
Done.


submissionid,assignmentid
1,1
2,2
3,3
4,4
5,5


In [11]:
%%sql
SELECT *
  FROM EvaluationRequest


 * sqlite:///gradedb.db
Done.


requestid,submissionid
1,1
2,4
3,2
4,5
5,3


In [10]:
%%sql
SELECT *
  FROM Evaluation

 * sqlite:///gradedb.db
Done.


evaluationid,requestid
1,1
2,2
3,3
4,4
5,5


In [12]:
  
%%sql
SELECT *
  FROM EvaluationFinished


 * sqlite:///gradedb.db
Done.


finishedid,evaluationid
1,1
2,2
3,3
4,4
5,5


In [13]:
%%sql
SELECT *
  FROM Scores

 * sqlite:///gradedb.db
Done.


scoreid,value,answerid,evaluationid
1,1,1,1
2,5,13,2
3,2,8,3
4,7,5,4
5,5,3,5


In [108]:
Stud_summ = text(''' SELECT 
    stu.universityid,
    stu.name AS 'Student Name',
    stu.email AS 'Student Email',
    tk.title AS 'Assignment',
    qu.title AS 'Question Title',
    qu.content AS 'Question Content',
    an.content AS 'Answer Content',
    sc.value AS 'Score'
  FROM Student stu
  LEFT JOIN Assignment asg ON asg.universityid = stu.universityid
  LEfT JOIN Task tk ON tk.taskid = asg.taskid
  LEFT JOIN Task_question tq ON tq.taskid = asg.taskid
  LEFT JOIN Question qu ON qu.questionid = tq.questionid
  LEFT JOIN Answers an ON an.questionid = tq.questionid
  LEFT JOIN Submission sb ON sb.assignmentid = asg.assignmentid
  LEFT JOIN EvaluationRequest er ON er.submissionid = sb.submissionid
  LEFT JOIN Evaluation ev ON ev.requestid = er.requestid
  LEFT JOIN EvaluationFinished ef ON ef.evaluationid = ev.evaluationid
  LEFT JOIN Scores sc ON sc.evaluationid = ev.evaluationid;
  ''')
  



In [109]:
df_Stud_summ = pd.read_sql(Stud_summ, engine)
df_Stud_summ

,universityid,Student Name,Student Email,Assignment,Question Title,Question Content,Answer Content,Score
0,S0416410,Marjorie Steele,0416410@vuw.leidenuniv.nl,Task 688,Question 1,Over infinitude squint cheerfulness curteous i...,Because coppi tiaralike aeromancer polyesters ...,7
1,S0416410,Marjorie Steele,0416410@vuw.leidenuniv.nl,Task 688,Question 5,Over awake temporize alienate mountain-quail N...,Resulting in papalistic folle undramatic accru...,7
2,S0416410,Marjorie Steele,0416410@vuw.leidenuniv.nl,Task 688,Question 8,Calculate differentials Wahabi whittret o.c. h...,None,7
3,S0225853,James Morse,0225853@vuw.leidenuniv.nl,Task 405,Question 6,How BASIL bloodshedding unplayability None None?,Given Salou vortices grainery clunking garnish...,7
4,S0225853,James Morse,0225853@vuw.leidenuniv.nl,Task 405,Question 6,How BASIL bloodshedding unplayability None None?,Given astrologize tassed unbefool specie sound...,7
5,S0225853,James Morse,0225853@vuw.leidenuniv.nl,Task 405,Question 6,How BASIL bloodshedding unplayability None None?,Given shinty handicraft pay-off gametocyte ove...,7
6,S0225853,James Morse,0225853@vuw.leidenuniv.nl,Task 405,Question 6,How BASIL bloodshedding unplayability None None?,Resulting in None cytopathies habiliments wadi...,7
7,S0225853,James Morse,0225853@vuw.leidenuniv.nl,Task 405,Question 6,How BASIL bloodshedding unplayability None None?,To calculate albuminuric stereoblind overweigh...,7
8,S0225853,James Morse,0225853@vuw.leidenuniv.nl,Task 405,Question 7,Over persay stopt Presb. repentingly augments?,Given ambrosine distopias uncurled intrahippoc...,7
9,S0225853,James Morse,0225853@vuw.leidenuniv.nl,Task 405,Question 10,How DMDA tenebrio flump constipation None?,Given provocatory cryptographically victualled...,7


In [110]:
def Student_Summary(StudentUniID):
    return(df_Stud_summ[df_Stud_summ['universityid']== StudentUniID])

In [111]:
Student_Summary('S1961057')

,universityid,Student Name,Student Email,Assignment,Question Title,Question Content,Answer Content,Score
11,S1961057,Dominic Robinson,1961057@vuw.leidenuniv.nl,Task 871,Question 2,Calculate incinerator None expounded under-kim...,To calculate invidia nobel Junr squashier trai...,7
12,S1961057,Dominic Robinson,1961057@vuw.leidenuniv.nl,Task 871,Question 3,Calculate methylomes e-cigarettes faciend fibe...,Because trammer saltpetrous shots miscategoris...,7
13,S1961057,Dominic Robinson,1961057@vuw.leidenuniv.nl,Task 871,Question 10,How DMDA tenebrio flump constipation None?,Given provocatory cryptographically victualled...,7
14,S1961057,Dominic Robinson,1961057@vuw.leidenuniv.nl,Task 871,Question 10,How DMDA tenebrio flump constipation None?,Resulting in unsew desolates lectures Batum hy...,7
